In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

In [2]:
def read_url(url):
    response = requests.get(url)
    page=response.text
    soup=BeautifulSoup(page,'lxml')
    return soup

# All apartments pages

In [3]:
url_list=[]

for i in range(1,357):
    url_list.append('https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/{}?rent_type=3'.format(i))

In [4]:
url_list[:5]

['https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/1?rent_type=3',
 'https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/2?rent_type=3',
 'https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/3?rent_type=3',
 'https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/4?rent_type=3',
 'https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/5?rent_type=3']

# Scrape apartment urls

In [5]:
apartments_list = []
for url in tqdm(url_list):
    for page in read_url(url).find_all('a',{'class':'listTitle'}):
        apartments_list.append('https://sa.aqar.fm' + page.get("href"))
len(apartments_list)

  0%|          | 0/356 [00:00<?, ?it/s]

7118

In [6]:
apartments_list[:5]

['https://sa.aqar.fm/شقق-للإيجار/الرياض/غرب-الرياض/حي-طويق/شارع-جبل-عسعس-حي-طويق-الرياض-الرياض-3367104',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الرياض/حي-القيروان/شارع-خليفه-العتيبي-حي-القيروان-الرياض-الرياض-3573103',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الرياض/حي-النرجس/طريق-الأمير-فيصل-بن-بندر-بن-عبدالعزيز-حي-النرجس-الرياض-الرياض-3563507',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الرياض/حي-الملقا/شارع-الحسيمة-حي-الملقا-الرياض-الرياض-3602889',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الرياض/حي-حطين/شارع-وادي-العمارية-حي-حطين-الرياض-الرياض-3582445']

write list to excel file

In [7]:
pd.Series(apartments_list).to_excel('apartments_list2.xlsx')

Or write it to a .txt file

In [8]:
textfile = open("apartments_list2.txt", "w")
for element in apartments_list:
    textfile.write(element + "\n")
textfile.close()

Read .txt file

In [9]:
with open('apartments_list2.txt','r') as f:
    apt_urls = f.read().splitlines()

In [10]:
len(apt_urls)

7118

In [11]:
apt_urls[:5]

['https://sa.aqar.fm/شقق-للإيجار/الرياض/غرب-الرياض/حي-طويق/شارع-جبل-عسعس-حي-طويق-الرياض-الرياض-3367104',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الرياض/حي-القيروان/شارع-خليفه-العتيبي-حي-القيروان-الرياض-الرياض-3573103',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الرياض/حي-النرجس/طريق-الأمير-فيصل-بن-بندر-بن-عبدالعزيز-حي-النرجس-الرياض-الرياض-3563507',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الرياض/حي-الملقا/شارع-الحسيمة-حي-الملقا-الرياض-الرياض-3602889',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الرياض/حي-حطين/شارع-وادي-العمارية-حي-حطين-الرياض-الرياض-3582445']

In [12]:
def convert(page):
    price_tag = page.find('span', {'class':'listingPagePrice'})
    price = float(price_tag.text.split()[4].replace(",",""))
    
    #if price_tag.text.split()[3][0]=='ش':
    #    return price * 12
    #elif price_tag.text.split()[3][0]=='ي':
    #    return price * 355
    #else:
    #    return price
    return price, price_tag.text.split()[3]

# Scrape pages details

In [13]:
PRICE_list = []
DISTRICT_list = []
BED_list = []
BATH_list = []
LIVING_List = []
AGE_list = []
FLOOR_list = []
TARGET_list = []
REGION_list = []
DURATION_list = []
urlapt = []

IMAGES_list = []

FURNISHED_list = []
KITCHEN_list = []
ANNEXE_list = []
PARKING_list = []
ELEVATOR_list = []
AC_list = []

R = ['وسط الرياض','جنوب الرياض','شمال الرياض','شرق الرياض','غرب الرياض']

for url in tqdm(apt_urls):
    try:
        # get page, price, neighborhood
        page = read_url(url)
        PRICE, DURATION = convert(page)

        DISTRICT = np.NaN
        div = page.find('div', class_='tree treeMargin')
        for link in div.find_all('a'):
            if(link.text.startswith('حي')):
                DISTRICT = link.text
            if any(r in link.text for r in R):
                REGION = link.text
        

        # main table
        table = page.find('table',{"class":"dc-detailsTable"}).find_all('tr')

        BED = np.NaN
        BATH = np.NaN
        AGE = np.NaN
        LIVING = np.NaN

        for tr in table:
            td = tr.find_all('td')
            row = [i.text for i in td]
            if row[1] == 'غرف النوم':
                BED = row[0]
            if row[1] == 'دورات مياه':
                BATH = row[0]
            if row[1] == 'الصالات':
                LIVING = row[0]
            if row[1] == 'عمر العقار':
                AGE = row[0].strip().split()[0]
                if AGE == 'جديد':
                    AGE = 0
            if row[1] == 'الدور':
                FLOOR = row[0]
            if row[1] == 'الفئة':
                if row[0] == "عوائل":
                    TARGET = "Families"
                if row[0] == "عزاب":
                    TARGET = "Bachelors"


        PRICE_list.append(PRICE)
        BED_list.append(BED)
        BATH_list.append(BATH)
        LIVING_List.append(LIVING)
        AGE_list.append(AGE)
        DISTRICT_list.append(DISTRICT)
        FLOOR_list.append(FLOOR)
        TARGET_list.append(TARGET)
        REGION_list.append(REGION)
        DURATION_list.append(DURATION)
        urlapt.append(url)

        # extra table
        FURNISHED, KITCHEN, ANNEXE, PARKING, ELEVATOR, AC = 0,0,0,0,0,0

        extraTable = page.find('table', {"class": "dc-extraDetailsTable"}).find_all('tr')

        for tr2 in extraTable:
            td2 = tr2.find_all('td')
            row2 = [i.text for i in td2]
            if 'مؤثثة' in row2:
                FURNISHED = 1
            if 'مطبخ' in row2:
                KITCHEN = 1
            if 'ملحق' in row2:
                ANNEXE = 1
            if 'مدخل سيارة' in row2:
                PARKING = 1
            if 'مصعد' in row2:
                ELEVATOR = 1
            if 'مكيفات' in row2:
                AC = 1


        FURNISHED_list.append(FURNISHED)
        KITCHEN_list.append(KITCHEN)
        ANNEXE_list.append(ANNEXE)
        PARKING_list.append(PARKING)
        ELEVATOR_list.append(ELEVATOR)
        AC_list.append(AC)        


        # image count
        try:            
            images = 0
            b = len(page.find('div', {'class':'listingImgContainer'}).findAll("img",{"class":"listingPageImg"}))
            c = len(page.find('div', {'class':'listingImgContainer'}).findAll("img",{"class":"listingPageImg hide"}))
            images = b-c
            IMAGES_list.append(images)
        except Exception:
            IMAGES_list.append(0)        


    except Exception as e:
        print(e, "->", url)

  0%|          | 0/7118 [00:00<?, ?it/s]

'NoneType' object has no attribute 'text' -> https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-الرمال/شارع-محمد-بن-محمد-النويري-حي-الرمال-الرياض-الرياض-3636194
'NoneType' object has no attribute 'text' -> https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-الرمال/شارع-محمد-بن-محمد-النويري-حي-الرمال-الرياض-الرياض-3636194
'NoneType' object has no attribute 'text' -> https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-الرمال/شارع-ابي-الفضل-الرصافي-حي-الرمال-الرياض-الرياض-3637034


In [14]:
dataset = pd.DataFrame({
    "DISTRICT": DISTRICT_list,
    "REGION": REGION_list,
    "PRICE": PRICE_list,
    "DURATION": DURATION_list,
    "BED": BED_list,
    "BATH": BATH_list,
    "LIVING": LIVING_List,
    "AGE": AGE_list,
    "IMAGES": IMAGES_list,
    "FLOOR": FLOOR_list,
    "TARGET": TARGET_list,
    "FURNISHED": FURNISHED_list,
    "KITCHEN": KITCHEN_list,
    "ANNEXE": ANNEXE_list,
    "PARKING": PARKING_list,
    "ELEVATOR": ELEVATOR_list,
    "AC": AC_list,
    "url": urlapt
})
dataset[:40]

,DISTRICT,REGION,PRICE,DURATION,BED,BATH,LIVING,AGE,IMAGES,FLOOR,TARGET,FURNISHED,KITCHEN,ANNEXE,PARKING,ELEVATOR,AC,url
0,حي طويق,غرب الرياض,1100.0,شهري,2,2,1,10,6,3,Families,0,1,0,0,0,0,https://sa.aqar.fm/شقق-للإيجار/الرياض/غرب-الري...
1,حي القيروان,شمال الرياض,30000.0,سنوي,2,1,1,0,9,علوي,Bachelors,1,1,0,1,1,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الر...
2,حي النرجس,شمال الرياض,45000.0,سنوي,3,2,1,0,7,3,Families,0,1,0,0,0,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الر...
3,حي الملقا,شمال الرياض,20000.0,سنوي,1,1,NaN,3,2,3,Families,0,1,0,0,1,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الر...
4,حي حطين,شمال الرياض,70000.0,سنوي,3,3,2,0,2,علوي,Families,0,1,0,0,1,0,https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الر...
5,حي الصحافة,شمال الرياض,20000.0,سنوي,1,1,NaN,2,4,علوي,Families,0,1,0,0,1,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الر...
6,حي اليرموك,شرق الرياض,19000.0,سنوي,1,1,1,6,4,3,Families,0,1,0,0,1,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الري...
7,حي ظهرة لبن,غرب الرياض,1000.0,شهري,1,1,1,0,2,علوي,Families,0,1,0,0,0,0,https://sa.aqar.fm/شقق-للإيجار/الرياض/غرب-الري...
8,حي الشهداء,شرق الرياض,21000.0,سنوي,1,1,1,NaN,8,علوي,Families,0,1,0,0,0,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الري...
9,حي الملقا,شمال الرياض,9000.0,سنوي,1,1,NaN,4,2,أرضي,Families,0,0,0,0,1,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الر...


In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7115 entries, 0 to 7114
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   DISTRICT   7115 non-null   object 
 1   REGION     7115 non-null   object 
 2   PRICE      7115 non-null   float64
 3   DURATION   7115 non-null   object 
 4   BED        7115 non-null   object 
 5   BATH       7115 non-null   object 
 6   LIVING     6444 non-null   object 
 7   AGE        5599 non-null   object 
 8   IMAGES     7115 non-null   int64  
 9   FLOOR      7115 non-null   object 
 10  TARGET     7115 non-null   object 
 11  FURNISHED  7115 non-null   int64  
 12  KITCHEN    7115 non-null   int64  
 13  ANNEXE     7115 non-null   int64  
 14  PARKING    7115 non-null   int64  
 15  ELEVATOR   7115 non-null   int64  
 16  AC         7115 non-null   int64  
 17  url        7115 non-null   object 
dtypes: float64(1), int64(7), object(10)
memory usage: 1000.7+ KB


In [16]:
dataset.to_excel("Aqarat3.xlsx")

Read excel file

In [17]:
d=pd.read_excel("Aqarat3.xlsx")
d.drop("Unnamed: 0",axis=1,inplace=True)
d

,DISTRICT,REGION,PRICE,DURATION,BED,BATH,LIVING,AGE,IMAGES,FLOOR,TARGET,FURNISHED,KITCHEN,ANNEXE,PARKING,ELEVATOR,AC,url
0,حي طويق,غرب الرياض,1100,شهري,2,2,1.0,10.0,6,3,Families,0,1,0,0,0,0,https://sa.aqar.fm/شقق-للإيجار/الرياض/غرب-الري...
1,حي القيروان,شمال الرياض,30000,سنوي,2,1,1.0,0.0,9,علوي,Bachelors,1,1,0,1,1,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الر...
2,حي النرجس,شمال الرياض,45000,سنوي,3,2,1.0,0.0,7,3,Families,0,1,0,0,0,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الر...
3,حي الملقا,شمال الرياض,20000,سنوي,1,1,NaN,3.0,2,3,Families,0,1,0,0,1,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الر...
4,حي حطين,شمال الرياض,70000,سنوي,3,3,2.0,0.0,2,علوي,Families,0,1,0,0,1,0,https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الر...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7110,حي الخليج,شرق الرياض,2300,شهري,1,1,1.0,0.0,0,علوي,Families,1,1,0,0,1,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الري...
7111,حي النرجس,شمال الرياض,80000,سنوي,3,2,1.0,3.0,0,أرضي,Families,1,1,0,1,0,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الر...
7112,حي اشبيلية,شرق الرياض,3300,شهري,1,1,1.0,2.0,0,علوي,Families,1,1,0,1,1,1,https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الري...
7113,حي البديعة,غرب الرياض,15000,شهري,3,2,1.0,10.0,0,أرضي,Families,0,1,0,0,0,0,https://sa.aqar.fm/شقق-للإيجار/الرياض/غرب-الري...


In [18]:
d.REGION.value_counts()

شرق الرياض     2721
شمال الرياض    2130
غرب الرياض     1427
جنوب الرياض     460
وسط الرياض      377
Name: REGION, dtype: int64